# Data Science ODL Project: Assessment 2

ID:

## Case study

Refer to the brief

## 1. Aims, objectives and plan (4 marks)

### a) Aims and objectives
A company has collected various attributes of the steel that they anneal, along with the attributes they have recorded the kind of annealing which was done previouly. 
They want to use these attributes and predict the type of Annealling that should be performed given a new instance of steel atrtibutes. 
The aim is develop a unbiased model which correctly predicts the annlealing class 98% of the times regardless of the class. In other words, if there 100 instances of each class, then the model 
should be able to correctly detect atleast 98 instances correctly from each class, which indicates that they want a model which has a very high True Positivity Rate.

###  b) Plan
Please demonstrate how you have conducted the project with a simple Gantt chart.

## 2. Understanding the case study (4 marks)

###  Case study analysis
State the key points that you found in the case and how you intend to deal with them appropriately to address the client's needs. (You can include more than four points.)


1. Overview of the Data
- The data is medium size with 798 training examples and 100 examples in test set. 
- There are 38 attributes, out which 6 are real-valued, 3 are oridnals and 29 categorical attributes. 
- The documentation of the data states that '-' represents the not_applicable values and '?' represent missing values. 
- Of the 29 categorical variables, 19 have binary values.
- There are 6 documentated annealing (target) types. '1', '2', '3', '4', '5' and 'U' 

2. Class Imbalance
    - Target distribution is heavily disblanced 
        - Class '3' dominates with 76% of the instances
        - Class '2' is present in  11.4% of instances
        - Class '5' is present in 7% of the instances
        - Class 'U' is present in 4% of the instances
        - Class '1' is present in 1% of the instances [only 8 examples]
        - Class '4' is not present in the data.
        - We will use synthetic minority over-sampling technique on the non-dominant classes.
            - N. V. Chawla, K. W. Bowyer, L. O.Hall, W. P. Kegelmeyer, "SMOTE: synthetic minority over-sampling technique," Journal of artificial intelligence research, 321-357, 2002.
        - Since class '1' has only 1% in training data and no presence in test set. 
            - We will ignore prediction of this class. 
            - With only 8 points our performance metrics will have, very little to NO significance. 
        - Class 4 has no instances in both train and test set, this class will be implicitly ignored.
        - Both these classes will require more data.


3. Missing Values
    - 9 attributes in the training set have <b> NO MISSING VALUES </b>.
    - Continous attributes [carbon, hardness, strength, thickm width, len] don't have any missing values in the train set.
        - But our pipeline will still have "impute with mean step" for these attributes to deal missingness during inference.
    - Out of 38 attributes 29 attributes have missing values. All of which are categorical or ordinal.
    - The amount of missingness varies from 8 % to 100%.
    - The data skewed in terms of missingness also, such that there are 4 variables 
        - ['steel', 'surface_quality', 'condition', 'formability'] with [8%, 27%, 33%, 35.4%] missingness, respectively.
        - Rest of the missing attributes have median missingness of 98% and a minimum of 76% missingness. 
    - To deal with this missing-ness we ran various experiments in the background.
        - Experiment 1 
            - We drop Drop all attributes/columns which have more that 35% missingness.
            - This leaves us with only 13 attributes, which is 34% of the original number attributes.
            - Imputation with mode of the training data, as all of them are categorical/ordinal.
       - Experiment 2
           - This experiment was insipired by 2 facts
               1. In experiment 1 we had dropped 66% of the attributes. Which is way too many dropped attributes, 25 in number. 
                   - There is a high chance that some of these attributes have high discrimatory power, wrt to the target.
               2. As mentioned above in the data documentation that:
                   - '-' represents the not_applicable values
                   - Of the 29 categorical variables, 19 have binary values.
                   - Except "shape", all other 18 attributes have very high missing values.
                   - Combining the above facts, it would makes a lot of sense to
                       - Impute missing values with  "not_applicable".
                   - However, we still drop attributes with more than 99% of missing values, which would be only 10 attributes.
                3. We would continue to impute ['steel', 'surface_quality', 'condition', 'formability'] with training data's mode in this experiment.
       - Experiment 3
           - Same as experiment 2, but we also impute ['steel', 'surface_quality', 'condition', 'formability'] with "not_applicable".
      
    
4. Since the client is interested in a high True Positivity Rate, we perform grid search based using F_beta, with more importace to recall, i.e. having the beta value set to 1.28. As per documentation of sklearn, a beta value higher than 1.0 prefers recall more than precision. 



In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from statistics import variance, mean
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, make_scorer, precision_score, fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import check_scoring
from sklearn.model_selection import cross_validate
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import _safe_indexing, check_random_state, shuffle
from scipy import sparse
from sklearn.ensemble._forest import ForestClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

## 3. Pre-processing applied (20 marks)
Enter the code in the cells below to execute each of the stated sub-tasks. 


#### Since the algorithms used in sklearn provide the functionality to internally convert string based categorical target to appropriate label encoding, we skip this step
#### Instead we present here the code to read the data file and get the train, val and test set.

In [12]:

def get_train_data():
    
    df = pd.read_csv("dataset/anneal.data")
    df = shuffle(df)
    
    # Because replace "?" represents missing. 
    df.replace("?", np.nan, inplace=True)

    # Because only 8 data poins have target as "1".
    # Our metrics won't be reliable for this class.
    
    df = df[df.target != "1"]
    
    # Just a fix as some values which are supposed to be int have string representation for this column
    df.enamelability = df.enamelability.astype(float)
    
    
    df_y = df.target
    df_X = df.drop(labels=["target"], axis=1)
    
    # Just re-organinsing the data such that countinous columns are at the end. 
    numerical_features = ["carbon", "hardness", "strength", "thick", "width", "len"]
    all_categorical_features = list(set(df_X.columns.to_list()) - set(numerical_features))
    df_X = pd.concat([df_X[all_categorical_features], df_X[numerical_features]], axis=1)
    
    return df_X, df_y

def get_test_data():
    
    df_test = pd.read_csv("dataset/anneal.test")
    df_test.replace("?", np.nan, inplace=True)
    
    df_test.enamelability = df_test.enamelability.astype(float)


    y_test = df_test.target
    X_test = df_test.drop(labels=["target"], axis=1)

    return X_test, y_test

###  b) Removing synonymous and noisy attributes if necessary 

##### All the attributes to remove will be added to a drop_features list
##### We will maintain different lists of drop attributes for various experiments as mentioned in the Case Study Section



In [13]:
X_train, y_train = get_train_data()

In [14]:
# Product Type attribute is the same throughout the dataset, hence it can removed
X_train.product_type.value_counts()

C    790
Name: product_type, dtype: int64

In [15]:
# Display amount of missingness in attributes
missing_means = X_train.isnull().mean().multiply(100).sort_values(ascending=False)
missing_means.head(10)

corr                      100.000000
m                         100.000000
s                         100.000000
exptl                     100.000000
jurofm                    100.000000
p                         100.000000
marvi                     100.000000
bc                         99.873418
blue_bright_varn_clean     99.493671
phos                       99.113924
dtype: float64

In [16]:
# Experiment 1 drop all attributes with more the 35% missing values
drop_attributes_exp1 = ["product_type"] + missing_means[missing_means > 35.0].index.to_list()

# Experiment 2 drop all attributes with more the 99% missing values
drop_attributes_exp2 = ["product_type"] + missing_means[missing_means > 99.0].index.to_list()

# Experiment 3; same as Exp 2.
drop_attributes_exp3 = drop_attributes_exp2

###### Column Dropper Transformer for Pipeline

In [17]:
class ColumnDropperTransformer:
    def __init__(self, column):
        self.columns = column

    def transform(self, X, y=None):
        X_new = X.drop(self.columns, axis=1)
        return X_new

    def fit(self, X, y=None):
        return self

###  c) Dealing with missing values if necessary 
##### We are dealing with missing values in three different ways as mentioned in Experiment 1, 2 and 3.

In [18]:
# Experiment 1: Impute missingness less than 35% with mode.
mode_imputer_list_exp1 = ["steel", "shape", "bore", "surface_quality", "formability"]

# Experiment 2: Impute missingness >75% with a not_applicable value/category. Impute missingness less than 35% with mode.
mode_imputer_list_exp2 = ["steel", "shape", "bore", "surface_quality", "formability"]
na_imputer_list_exp2 = missing_means[(missing_means > 70.0) & (missing_means < 99.0)].index.to_list()

# Experiment 3: Impute all missing value with NA category.
na_imputer_list_exp3 = missing_means[(missing_means > 0.0) & (missing_means < 99.0)].index.to_list()

###### NA imputer Transformer for Pipeline

In [19]:
class NAColumnTransformer:
    def __init__(self, column):
        self.columns = column

    def transform(self, X, y=None):
        
        without_ordinals = list(set(self.columns) - {'formability', 'enamelability'})
        
        # Imputer string based features with string NA
        X_new = X[without_ordinals].fillna("NA")
        X[without_ordinals] = X_new
        
        # Creating a new 0 category for formability
        # Creating a new 0 category for enamelability
        if "formability" in self.columns:
            X['formability'] = X['formability'].fillna(0).astype(int)

        if 'enamelability' in self.columns:
            X['enamelability'] = X['enamelability'].fillna(0).astype(int)
        return X

    def fit(self, X, y=None):
        return self


###  d) Rescaling if necessary
### For Random Forest Scale is kept False as it can handle unscaled values.

In [20]:
def get_numerical_pipeline(scale=False):
    if scale:
        numerical_pipeline = Pipeline(steps=[('ss', StandardScaler())])
        return numerical_pipeline

    return 'passthrough'

### e) Categorical Feature Pipeline


- in case of experiment 3, simple imputer will not have any effect
- as all the `nan` values would be imputed by na_imputer.

In [21]:
def get_categorical_pipeline(cat_features_to_drop, na_imputer_cols):

    categorical_pipeline = Pipeline(steps=[
        ('drop_column', ColumnDropperTransformer(cat_features_to_drop)),
        ('na_imputer', NAColumnTransformer(na_imputer_cols)),
        ('mode', SimpleImputer(strategy='most_frequent')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    return categorical_pipeline

### f)  Full Preprocessor Pipeline

In [22]:
def get_full_processeror(all_categorical_features, cat_features_to_drop, numerical_features, na_imputer_cols, scale_numerical):
    """
    
    :param all_categorical_features: List of all the categorical features.
    :param cat_features_to_drop: List of categorical features to drop.
    :param numerical_features: List of numerical features.
    :param na_imputer_cols: List of features to imputer with new "not_applicable" category. 
    :param scale_numerical: Boolean which controls scaling of numerical features.
    :return: 
    """
    
    categorical_pipeline = get_categorical_pipeline(cat_features_to_drop, na_imputer_cols)
    numerical_pipeline = get_numerical_pipeline(scale=scale_numerical)
    full_processor = ColumnTransformer(transformers=[
        ('category', categorical_pipeline, all_categorical_features),
        ('numerical', numerical_pipeline, numerical_features)
    ])
    return full_processor

In [23]:
all_features = X_train.columns.to_list()
numerical_features = ["carbon", "hardness", "strength", "thick", "width", "len"]
all_categorical_features = list(set(all_features) - set(numerical_features))

#### Experiment 1 Data Pipeline
- We drop Drop all attributes/columns which have more that 35% missingness.
- We impute categorical values with mode.

In [24]:
def get_experiment1_pipeline(scale_numerical_features=False):
    experiment1_data_pipeline = get_full_processeror(
        all_categorical_features=all_categorical_features, 
        cat_features_to_drop=drop_attributes_exp1, 
        numerical_features=numerical_features, 
        na_imputer_cols=[], 
        scale_numerical=scale_numerical_features)
    return experiment1_data_pipeline

#### Experiment 2 Data Pipeline
- We drop Drop all attributes/columns which have more that 99% missingness.
- We impute ["steel", "shape", "bore", "surface_quality", "formability"] categorical attributes with mode.
- Other categorical attributes are imputed with "NA". Explaination is given in the case study section.

In [25]:
def get_experiment2_pipeline(scale_numerical_features=False):
    experiment2_data_pipeline = get_full_processeror(
        all_categorical_features=all_categorical_features, 
        cat_features_to_drop=drop_attributes_exp3, 
        numerical_features=numerical_features, 
        na_imputer_cols=na_imputer_list_exp2, 
        scale_numerical=scale_numerical_features)
    return experiment2_data_pipeline

#### Experiment 3 Data Pipeline
- We drop Drop all attributes/columns which have more that 99% missingness.
- All categorical attributes are imputed with "NA". Explaination is given in the case study section.

In [26]:
def get_experiment3_pipeline(scale_numerical_features=False):
    experiment3_data_pipeline = get_full_processeror(
        all_categorical_features=all_categorical_features, 
        cat_features_to_drop=drop_attributes_exp3, 
        numerical_features=numerical_features, 
        na_imputer_cols=na_imputer_list_exp3, 
        scale_numerical=scale_numerical_features)
    return experiment3_data_pipeline


## 4. Technique 1 (20 marks)

### a) Discuss your motivation for choosing the technique and provide a schematic figure of the process

We use Random Forest as first technique because of the following motivations:

1. The algorithm reduces the overfitting and variance problem by using bagging and ensembling.
    - It tackles overfitting and variance by builds many decision trees with subsets of features[bagging].
    - It then takes the majority vote of the outputs of all trees to give the final output. 
2. Reqiures no features scaling. 
3. Robustness towards outliers.
4. The grid search is easy to perform.
    - Unlike paramteric models, doesn't require careful tuning of regularisation parameter, as regularisation is in-built because of ensembling and bagging.

### Handling Oversampling
As mentioned in the case study section, the target distribution is heavily skewed. 
To tackle this we employ SMOTE based oversampling.
The following code is insipired by:
- The SMOTENC class in [imbalance-learn](https://github.com/scikit-learn-contrib/imbalanced-learn/blob/6176807c9c5d68126a79771b6c0fce329f632d2f/imblearn/over_sampling/_smote/base.py#L368) library 
- N. V. Chawla, K. W. Bowyer, L. O.Hall, W. P. Kegelmeyer, "SMOTE: synthetic minority over-sampling technique," Journal of artificial intelligence research, 321-357, 2002.

In [27]:
class Oversample:
    def __init__(self, continuous_features_indices, sampling_strategy):
        self.continuous_features_ = continuous_features_indices
        self.sampling_strategy_ = sampling_strategy

    def resample(self, X, y=None):
        self.n_features_ = X.shape[1]
        target_stats = Counter(y)
        class_minority = min(target_stats, key=target_stats.get)
        X_continuous = X[:, self.continuous_features_]
        X_minority = _safe_indexing(X_continuous, np.flatnonzero(y == class_minority))

        self.categorical_features_ = [i for i in range(len(self.continuous_features_), self.n_features_)]
        X_categorical = X[:, self.categorical_features_]
        self.ohe_ = OneHotEncoder(handle_unknown="ignore", dtype=np.float64, sparse=False)
        X_ohe = self.ohe_.fit_transform(X_categorical)


        var = X_minority.var(axis=0)
        median_std_ = np.median(np.sqrt(var))
        # Categorical attributes

        X_encoded_pre = np.hstack((X_continuous, X_ohe))

        X_resampled = [X_encoded_pre]
        y_resampled = [y.copy()]

        X_categorical_transformed = X_ohe * median_std_ / 2

        X_encoded = np.hstack((X_continuous, X_categorical_transformed))

        for class_sample, n_samples in self.sampling_strategy_.items():  
            target_class_indices = np.flatnonzero(y == class_sample)
            X_class = _safe_indexing(X_encoded, target_class_indices)
            nn = NearestNeighbors(n_neighbors=6)
            # print("looped")

            nn.fit(sparse.csr_matrix(X_class))
            nns = nn.kneighbors(sparse.csr_matrix(X_class), return_distance=False)[:, 1:]

            n_samples_gen = n_samples-X_class.shape[0]
            X_new, y_new = self.gen_data(X_class, y.dtype, class_sample, nns, n_samples_gen)
            X_resampled.append(X_new)
            y_resampled.append(y_new)

        X_resampled = np.vstack(X_resampled)
        y_resampled = np.hstack(y_resampled)

        # reverse the encoding of the categorical features
        X_res_cat = X_resampled[:, len(self.continuous_features_):]
        X_res_cat_dec = self.ohe_.inverse_transform(X_res_cat)

        X_resampled = np.hstack(
            (
                X_resampled[:, : len(self.continuous_features_)],
                X_res_cat_dec,
            )
        )

        return X_resampled, y_resampled

    def gen_data(self, X, y_dtype, y_type, nearest_neighours, n_samples):
        random_state = check_random_state(None)
        samples_indices = random_state.randint(low=0, high=nearest_neighours.size, size=n_samples)
        rows = np.floor_divide(samples_indices, nearest_neighours.shape[1])
        cols = np.mod(samples_indices, nearest_neighours.shape[1])

        diffs = X[nearest_neighours[rows, cols]] - X[rows]  
        X_new = X[rows] + random_state.uniform(size=n_samples)[:, np.newaxis] * diffs

        all_neighbors = X[nearest_neighours[rows]] 

        categories_size = [len(self.continuous_features_)] + [cat.size for cat in self.ohe_.categories_]

        for start_idx, end_idx in zip(
                np.cumsum(categories_size)[:-1], np.cumsum(categories_size)[1:]
        ):
            col_maxs = all_neighbors[:, :, start_idx:end_idx].sum(axis=1)
            # tie breaking argmax
            is_max = np.isclose(col_maxs, col_maxs.max(axis=1, keepdims=True))
            max_idxs = random_state.permutation(np.argwhere(is_max))
            xs, idx_sels = np.unique(max_idxs[:, 0], return_index=True)
            col_sels = max_idxs[idx_sels, 1]
            ys = start_idx + col_sels
            X_new[:, start_idx:end_idx] = 0
            X_new[xs, ys] = 1

        y_new = np.full(n_samples, fill_value=y_type, dtype=y_dtype)
        return X_new, y_new

In [28]:
numericals_indices = [i for i in range(6)]
oversample = Oversample(continuous_features_indices=numericals_indices,
                    sampling_strategy={'2': 100, '5': 100, 'U': 120})

#### We slightly modify Random Forest and redefine the fit method, such that before calling fit we over sample the data.
This ensures that only the train set it oversamples, as test set set is only fed to the predict method.


In [29]:
class RandomForestClassifierCustom(ForestClassifier):

    def __init__(
            self,
            n_estimators=100,
            *,
            criterion="gini",
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            min_weight_fraction_leaf=0.0,
            max_features="sqrt",
            max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            bootstrap=True,
            oob_score=False,
            n_jobs=None,
            random_state=None,
            verbose=0,
            warm_start=False,
            class_weight=None,
            ccp_alpha=0.0,
            max_samples=None,
            oversample=None
    ):
        super().__init__(
            base_estimator=DecisionTreeClassifier(),
            n_estimators=n_estimators,
            estimator_params=(
                "criterion",
                "max_depth",
                "min_samples_split",
                "min_samples_leaf",
                "min_weight_fraction_leaf",
                "max_features",
                "max_leaf_nodes",
                "min_impurity_decrease",
                "random_state",
                "ccp_alpha",
            ),
            bootstrap=bootstrap,
            oob_score=oob_score,
            n_jobs=n_jobs,
            random_state=random_state,
            verbose=verbose,
            warm_start=warm_start,
            class_weight=class_weight,
            max_samples=max_samples,
        )

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes
        self.min_impurity_decrease = min_impurity_decrease
        self.ccp_alpha = ccp_alpha
        self.oversample = oversample

    def fit(self, X, y, sample_weight=None):
        X, y = self.oversample.resample(X, y)
        super().fit(X, y)

Enter the correct code in the cells below to execute each of the stated sub-tasks.
### b) Setting hyper parameters with rationale


###### We perform a grid search on number of estimators (decision trees) and max depth of those decision trees.
###### The number of estimators will tackle variance problem and but at the same time will increase the time complexity of the algorithm.
###### We tune the max depth parameter casue very deep descision trees cause overfitting, but literature also suggests that, we can allow the decision trees to grow as deep as possible as long as the ensemble is large enough.


In [30]:
rf_param_grid = {
        "model__n_estimators": [20, 40, 60, 80, 100, 120, 150, 200],
        "model__max_depth": [5, 10, 20, 30, 40, None],

    }

### c) Performance metrics for training

In [31]:
def recall_biased_Fbeta(y_true, y_pred):
    fbs = fbeta_score(y_true, y_pred, average='macro', beta=1.28)
    return fbs

def print_recall_biased_Fbeta(y_true, y_pred):
    print(classification_report(y_true, y_pred, zero_division=0))  # print classification report
    fbs = fbeta_score(y_true, y_pred, average='macro', beta=1.28)
    print("F1_beta: ", fbs)
    print("\n***---***\n")
    return fbs

### d) Optimising hyper parameters
We perform nested cross validation along with grid search, and return the best estimators. 
Instead of running nested CV only once, we perform many trials of it and record the best estimators on each fold, this is because the 
during nested CV the best estimator on each fold might be different, so we keep track of the best estimators during many runs and pick the 
top 3 estimators by frequency. 

These top 3 estimators will then be ensembled again to get the final classifier.

In [97]:
def nested_cross_validation(X_train, y_train, param_grid, classifier, datapipeline, rf_or_lr):
    """
    X_train, y_train: Training Data
    param_grid: parameter grid
    classifier: classifier used.
    datapipeline: datapipeline of the experiment
    rf_or_lr: is the classifier RandomForest or Linear Regression. Helps in recoding the winning parameters.
    """
    n_jobs = 40

    pipeline = Pipeline(steps=[
        ('preprocess', datapipeline),
        ('model', classifier)
    ])

    clf = GridSearchCV(estimator=pipeline, param_grid=param_grid,
                       scoring=make_scorer(recall_biased_Fbeta), cv=3, n_jobs=n_jobs, return_train_score=True)

    
    scorer = check_scoring(clf, scoring=make_scorer(print_recall_biased_Fbeta))
    cv_results = cross_validate(
        estimator=clf,
        X=X_train,
        y=y_train,
        scoring={"score": scorer},
        cv=3,
        n_jobs=n_jobs,
        pre_dispatch="2*n_jobs",
        error_score=np.nan,
        return_estimator=True,
        return_train_score=True
    )

    run_test_Fbeta_score_mean = mean(cv_results['test_score'])
    run_train_Fbeta_score_mean = mean(cv_results['train_score'])
    
    # run_var = variance(cv_results['test_score'])
    mean_train_Fbeta_scores.append(run_train_Fbeta_score_mean)
    mean_test_Fbeta_scores.append(run_test_Fbeta_score_mean)
    
    for estimator in cv_results['estimator']:
        if rf_or_lr == 'rf':
            estimator_frequency[(estimator.best_estimator_.named_steps.model.max_depth, estimator.best_estimator_.named_steps.model.n_estimators)] += 1
        elif rf_or_lr == 'lr':
            estimator_frequency[(estimator.best_estimator_.named_steps.model.penalty, estimator.best_estimator_.named_steps.model.C)] += 1


#### Experiment 1

In [88]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
rf = RandomForestClassifierCustom(oversample=oversample, n_jobs=40, oob_score=False)
exp1_pipeline = get_experiment1_pipeline()
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), rf_param_grid, rf, exp1_pipeline, rf_or_lr='rf')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
rf_experiment1_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]")
print(rf_experiment1_best_estimators)

MEAN TRAIN FBeta:  0.994385574519773 MEAN TEST FBeta:  0.9032225328187478
TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]
[('l2', 10), ('l1', 15), ('l1', 10)]


#### Experiment 2

In [91]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
rf = RandomForestClassifierCustom(oversample=oversample, n_jobs=40, oob_score=False)
exp2_pipeline = get_experiment2_pipeline()
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), rf_param_grid, rf, exp2_pipeline, rf_or_lr='rf')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
rf_experiment2_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]")
print(rf_experiment2_best_estimators)

MEAN TRAIN FBeta:  0.9979216619053568 MEAN TEST FBeta:  0.9399244615810068
TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]
[(20, 20), (40, 150), (40, 40)]


#### Experiment 3

In [90]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
rf = RandomForestClassifierCustom(oversample=oversample, n_jobs=40, oob_score=False)
exp3_pipeline = get_experiment3_pipeline()
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), rf_param_grid, rf, exp3_pipeline, rf_or_lr='rf')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
rf_experiment3_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]")
print(rf_experiment3_best_estimators)

MEAN TRAIN FBeta:  1.0 MEAN TEST FBeta:  0.9827723978919465
TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]
[(20, 60), (10, 40), (40, 150)]


### Performance on Test Set

#### Data pipeline from experiment 3 shows the best result in cross validation, hence we should reccomend this model pipeline.
#### We still call the `rf_final_results` on all the experiment pipelines, just for comapisons purpose.
#### Re-fitting  Ensembling top 3 random forest reported by nested cross validation and calculating scores on Test Set

During nested cross validation, different models/parameters win on different folds of the data.
So, in the above 10 trials of nested cross validation we kept track of the winning parameters for Random Forest
and now we make an ensemble of the top three random forests and caluculate the scores on the test set to show the finals results.

In [124]:
def rf_final_results(pipeline_method, max_depth_n_est_list):
    """
    pipeline_method: the method to get pipeline of the experiment. eg: get_experiment2_pipeline
    max_depth_n_est_list: list of top 3 estimators from nested cv. The first parameter is 
    """
    n_jobs = 40
    numericals_indices = [i for i in range(6)]
    oversample_final = Oversample(continuous_features_indices=numericals_indices,
                                  sampling_strategy={0: 100, 2: 100, 3: 120})
    rf1 = RandomForestClassifierCustom(oversample=oversample_final, max_depth=max_depth_n_est_list[0][0], n_estimators=max_depth_n_est_list[0][1], n_jobs=n_jobs,
                                       oob_score=False)
    rf2 = RandomForestClassifierCustom(oversample=oversample_final, max_depth=max_depth_n_est_list[1][0], n_estimators=max_depth_n_est_list[1][1], n_jobs=n_jobs,
                                       oob_score=False)
    rf3 = RandomForestClassifierCustom(oversample=oversample_final, max_depth=max_depth_n_est_list[2][0], n_estimators=max_depth_n_est_list[2][1], n_jobs=n_jobs,
                                       oob_score=False)
    clfHard = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3)], voting='hard')
    X_test, y_test = get_test_data()
    final_data_pipeline = pipeline_method()
    rf_pipeline = Pipeline(steps=[
        ('preprocess', final_data_pipeline),
        ('model', clfHard)
    ])
    rf_pipeline.fit(X_train, y_train)
    y_pred = rf_pipeline.predict(X_test)
    print("RF TEST SET REPORT")
    print(classification_report(y_test, y_pred, zero_division=0))
    print("-"*60)
    print("RF TRAIN SET REPORT")
    y_train_pred = lr_pipeline.predict(X_train)
    print(classification_report(y_train, y_train_pred, zero_division=0))

### Results from Experiment 1

In [125]:
rf_final_results(pipeline_method=get_experiment1_pipeline, max_depth_n_est_list=rf_experiment1_best_estimators)

RF TEST SET REPORT
              precision    recall  f1-score   support

           2       0.79      1.00      0.88        11
           3       1.00      0.95      0.97        76
           5       0.88      1.00      0.93         7
           U       1.00      1.00      1.00         6

    accuracy                           0.96       100
   macro avg       0.92      0.99      0.95       100
weighted avg       0.97      0.96      0.96       100

------------------------------------------------------------
RF TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.97      0.97      0.97        88
           3       0.99      1.00      0.99       608
           5       1.00      1.00      1.00        60
           U       1.00      0.94      0.97        34

    accuracy                           0.99       790
   macro avg       0.99      0.98      0.98       790
weighted avg       0.99      0.99      0.99       790



### Results from Experiment 2

In [126]:
rf_final_results(pipeline_method=get_experiment2_pipeline, max_depth_n_est_list=rf_experiment2_best_estimators)

RF TEST SET REPORT
              precision    recall  f1-score   support

           2       0.92      1.00      0.96        11
           3       1.00      0.99      0.99        76
           5       1.00      1.00      1.00         7
           U       1.00      1.00      1.00         6

    accuracy                           0.99       100
   macro avg       0.98      1.00      0.99       100
weighted avg       0.99      0.99      0.99       100

------------------------------------------------------------
RF TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.97      0.97      0.97        88
           3       0.99      1.00      0.99       608
           5       1.00      1.00      1.00        60
           U       1.00      0.94      0.97        34

    accuracy                           0.99       790
   macro avg       0.99      0.98      0.98       790
weighted avg       0.99      0.99      0.99       790



### Results from Experiment 3

In [127]:
rf_final_results(pipeline_method=get_experiment3_pipeline, max_depth_n_est_list=rf_experiment3_best_estimators)

RF TEST SET REPORT
              precision    recall  f1-score   support

           2       1.00      1.00      1.00        11
           3       1.00      1.00      1.00        76
           5       1.00      1.00      1.00         7
           U       1.00      1.00      1.00         6

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

------------------------------------------------------------
RF TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.97      0.97      0.97        88
           3       0.99      1.00      0.99       608
           5       1.00      1.00      1.00        60
           U       1.00      0.94      0.97        34

    accuracy                           0.99       790
   macro avg       0.99      0.98      0.98       790
weighted avg       0.99      0.99      0.99       790



## 5. Technique 2 (20 marks)

### a) Discuss your motivation for choosing the technique and  provide a schematic figure of the process

Second we choose Logistic Regression without feature mapping. Logistic Regression is that it is one the simplest parametric model for classification and using it without feature mapping further simplifies the model. Primary reason of choosing this combination is applying the simplest hypothesis to the data. (Occam's Razor)

We use `liblinear` as the solver because it allows both `l1` and `l2` penalty terms


#### We overload the Logistic regression class to oversample while training. 

In [68]:
numericals_indices = [i for i in range(6)]
oversample_lr = Oversample(continuous_features_indices=numericals_indices,
                    sampling_strategy={'2': 150, '5': 120, 'U': 100})

In [69]:
class LogisticRegressionCustom(LogisticRegression):

    def __init__(self, oversample, solver, penalty=None, C=None):
        super().__init__(penalty=penalty, C=C, solver=solver)
        self.oversample = oversample

    def fit(self, X, y, sample_weight=None):
        X, y = self.oversample.resample(X, y)
        super().fit(X, y)

### b) Setting hyper parameters with rationale

For Logistic Regression we intend on tuning the penalty type, 'l1' and 'l2' and these have direct relation with the final weights/paramters that are learnt by the model.
- L1: Produces sparse weights.
- L2: Doesn't push less important features to 0.

Second we tune the C parameter, which controls regularisation strength. 
- Smaller values of C specify stronger regularization.

In [70]:
lr_param_grid = {
        "model__penalty": ['l1', 'l2'],
        "model__C": [0.5, 1, 5, 10, 15, 20],
    }

### c) Optimising hyper parameters

#### Experiment 1

In [98]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
lr = LogisticRegressionCustom(oversample=oversample_lr, solver='liblinear')
exp1_pipeline = get_experiment1_pipeline(scale_numerical_features=True)
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), lr_param_grid, lr, exp1_pipeline, rf_or_lr='lr')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
experiment1_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]")
print(experiment1_best_estimators)

MEAN TRAIN FBeta:  0.6823245534669126 MEAN TEST FBeta:  0.6359302379085366
TOP 3 estimators [0th index is max_depth and 1st index is n_estimators]
[('l2', 15), ('l1', 15), ('l2', 20)]


#### Experiment 2

In [99]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
lr = LogisticRegressionCustom(oversample=oversample_lr, solver='liblinear')
exp2_pipeline = get_experiment2_pipeline(scale_numerical_features=True)
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), lr_param_grid, lr, exp2_pipeline, rf_or_lr='lr')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
experiment2_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is penalty and 1st index is C]")
print(experiment2_best_estimators)

MEAN TRAIN FBeta:  0.8638307584123754 MEAN TEST FBeta:  0.8294546195945323
TOP 3 estimators [0th index is penalty and 1st index is C]
[('l1', 20), ('l1', 15), ('l1', 5)]


#### Experiment 3

In [100]:
mean_train_Fbeta_scores = []
mean_test_Fbeta_scores = []
estimator_frequency = defaultdict(int)
lr = LogisticRegressionCustom(oversample=oversample_lr, solver='liblinear')
exp3_pipeline = get_experiment3_pipeline(scale_numerical_features=True)
for i in range(5):
    nested_cross_validation(X_train.copy(), y_train.copy(), lr_param_grid, lr, exp3_pipeline, rf_or_lr='lr')
print("MEAN TRAIN FBeta: ", mean(mean_train_Fbeta_scores), "MEAN TEST FBeta: ", mean(mean_test_Fbeta_scores))
experiment3_best_estimators = [x[0] for x in list(sorted(estimator_frequency.items(), key=lambda item: item[1], reverse=True))[:3]]
print("TOP 3 estimators [0th index is penalty and 1st index is C]")
print(experiment3_best_estimators)

MEAN TRAIN FBeta:  0.9774212941057975 MEAN TEST FBeta:  0.954270098434053
TOP 3 estimators [0th index is penalty and 1st index is C]
[('l1', 20), ('l1', 5), ('l1', 15)]


### We choose `l1` penalty along with `20` as the `C value` and the `experiment 3's` pipeline.

### d) Performance metrics on Test Set

In [120]:
def LR_test_set_performance(data_pipeline):
    lr_final = LogisticRegressionCustom(oversample=oversample_lr, solver='liblinear', penalty='l1', C=20)
    X_test, y_test = get_test_data()
    final_data_pipeline = get_experiment3_pipeline()
    lr_pipeline = Pipeline(steps=[
        ('preprocess', data_pipeline),
        ('model', lr_final)
    ])
    lr_pipeline.fit(X_train, y_train)
    y_pred = lr_pipeline.predict(X_test)
    print("LR TEST SET REPORT")
    print(classification_report(y_test, y_pred, zero_division=0))
    print("-"*60)
    print("LR TRAIN SET REPORT")
    y_train_pred = lr_pipeline.predict(X_train)
    print(classification_report(y_train, y_train_pred, zero_division=0))

#### LR Experiment 1

In [121]:
lr_exp1_data_pipeline = get_experiment1_pipeline()
LR_test_set_performance(lr_exp1_data_pipeline)

LR TEST SET REPORT
              precision    recall  f1-score   support

           2       0.67      0.73      0.70        11
           3       0.95      0.92      0.93        76
           5       0.78      1.00      0.88         7
           U       1.00      0.83      0.91         6

    accuracy                           0.90       100
   macro avg       0.85      0.87      0.85       100
weighted avg       0.91      0.90      0.90       100

------------------------------------------------------------
LR TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.68      0.44      0.54        88
           3       0.89      0.94      0.92       608
           5       0.76      0.78      0.77        60
           U       0.96      0.74      0.83        34

    accuracy                           0.87       790
   macro avg       0.82      0.73      0.76       790
weighted avg       0.86      0.87      0.86       790



#### LR Experiment 2

In [122]:
lr_exp2_data_pipeline = get_experiment2_pipeline()
LR_test_set_performance(lr_exp2_data_pipeline)

LR TEST SET REPORT
              precision    recall  f1-score   support

           2       0.75      0.82      0.78        11
           3       0.97      0.96      0.97        76
           5       1.00      1.00      1.00         7
           U       1.00      1.00      1.00         6

    accuracy                           0.95       100
   macro avg       0.93      0.94      0.94       100
weighted avg       0.95      0.95      0.95       100

------------------------------------------------------------
LR TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.73      0.56      0.63        88
           3       0.93      0.97      0.95       608
           5       1.00      1.00      1.00        60
           U       0.97      0.91      0.94        34

    accuracy                           0.92       790
   macro avg       0.91      0.86      0.88       790
weighted avg       0.92      0.92      0.92       790



#### LR Experiment 3

In [123]:
lr_exp3_data_pipeline = get_experiment3_pipeline()
LR_test_set_performance(lr_exp3_data_pipeline)

LR TEST SET REPORT
              precision    recall  f1-score   support

           2       0.92      1.00      0.96        11
           3       1.00      0.99      0.99        76
           5       1.00      1.00      1.00         7
           U       1.00      1.00      1.00         6

    accuracy                           0.99       100
   macro avg       0.98      1.00      0.99       100
weighted avg       0.99      0.99      0.99       100

------------------------------------------------------------
LR TRAIN SET REPORT
              precision    recall  f1-score   support

           2       0.97      0.95      0.96        88
           3       0.99      1.00      0.99       608
           5       1.00      1.00      1.00        60
           U       1.00      0.94      0.97        34

    accuracy                           0.99       790
   macro avg       0.99      0.97      0.98       790
weighted avg       0.99      0.99      0.99       790



## 6. Comparison of metrics performance for testing (16 marks) 


### a) Use of cross validation for both techniques to deal with over-fitting

- The method `nestes_cross_validation` defined above is the method used to perform Nested Cross Validation.  
- Nested cross validation was used to get a robust validation score and get best parameters through grid search.
- The challenge of using oversampling along with nested cross validation was dealt with by modifying the fit method by using Overloading of Random Forest and Logistic Regression Classes.

### b) Comparison with appropriate metrics for testing

- We have used classification report which conveys Precision, Recall, F1 and Accuracy.
- Such a report conveys all the important metrics about the predicitons of the classes. 

### c) Model selection (ROC or other charts)

- We used classfication reports, which convery that the Random Forest outperform Logistic Regression on the test set.

## 7. Final recommendation of best model (8 marks)

### a) Discuss the results from a technical perspective, for example, overfitting discussion, complexity and efficiency

- Both RF and LR have the similar performance, however Random Forest Outperforms LR.
- Since RF correctly predicted all the test set instances, it conveys that the test set has easy instances. 
    - Also, classes '2', '5', 'U' have only 11, 7 and 6 instances.
    - So, the test set metrics might not be so reliable.
    - Such good performance could be just luck, small test set size and easy test set.
- Overfitting:
    - We saw signs of overfitting with the Experiment 1's pipeline. 
        - We had removed over `60%` of the features in experiment 1.
        - MEAN TRAIN FBeta:  `0.994` 
        - MEAN TEST FBeta:  `0.903`
    - Overfitting was reduced when we started using more features in Experiment 2 and Experiment 3.
        - Experiment 2
            - MEAN TRAIN FBeta:  `0.997` 
            - MEAN TEST FBeta:  `0.939`
        - Experiment 3
            - MEAN TRAIN FBeta:  `1.0` 
            - MEAN TEST FBeta:  `0.982`
- Underfitting:
    - We saw signs of underfitting when we used LR with the less features in Experiment 1
        - MEAN TRAIN FBeta:  0.6823245534669126 
        - MEAN TEST FBeta:  0.6359302379085366
        - Also, with less features we observed model preferred l2 penalty.
- Complexity
    - With Random Forest we let the individual decision trees grow as deep as possible which increases complexity but since we use bagging and ensembling along with it, it combats overffitng. 

### Our final recommendation of best model with the given data set would be 
- An Ensemble of Random Forest along with the data pipeline used in experiment 3, which imputes the missing values with a new category. 
- And we would conduct the training with over sampling using SMOTE. 
- We chose RF not only because of it's perfect result but also becuase of the nature of the data and problem, a Rule Based (Decision Tree) based classifier would be appropriate. 
- Also, Random Forest combats overfitting, outliers better than Logistic Regression.

### b) Discuss the results from a business perspective, for example, results interpretation, relevance and balance with technical perspective

- The classification report shows that the results on the test set are perfect, but we should take into account that the test was of only 100 instance, where except the majority class, the other classes have very small representation.

## 8. Conclusion (8 marks)

### a) What has been successfully accomplished and what has not been successful?

- We have been able to succcesfully train 2 models, Random Forest and Logistic Regression. 
- We experimented with various imputation methods. 
- We finally were able to decide on recommending the Random Forest classifier. Because of 
    - The perfect result on the test set.
    - Nature of the problem and the data. 
- Also our model meets the client expectation of more than 98% accuracy for each class. 

### b) Reflecting back on the analysis, what could you have done differently if you were to do the project again?

- We believe that the test set is too small for any concrete conclusion of the performance of the classifiers.
- Also it could be the case the test set might be too easy.
- As a recommendation to the firm, I'd ask them to collect more test data, specially corersponding to the non-majority classes.

- The data set might have some problem because  to be a problem because 

While reading https://archive.ics.uci.edu/ml/datasets/Annealing, the data documenation mentions 

The '-' values are actually 'not_applicable' values rather than 'missing_values' (and so can be treated as legal discrete values rather than as showing the absence of a discrete value).

But we don't find any '-' in the data, however, imputing with 'not_applicable' values gave the best results. 